In [1]:
import fnmatch
import json
import getpass
import os
import pathlib
import datetime
import laspy


import time
import requests
                    
from dask.distributed import LocalCluster, SSHCluster 
from laserfarm import Retiler, DataProcessing, GeotiffWriter, MacroPipeline
from laserfarm.remote_utils import get_wdclient, get_info_remote, list_remote

## Global Configuration

In [2]:
import fnmatch
import json
import getpass
import os
import pathlib
import datetime
                    
from dask.distributed import LocalCluster, SSHCluster 
from laserfarm import Retiler, DataProcessing, GeotiffWriter, MacroPipeline
from laserfarm.remote_utils import get_wdclient, get_info_remote, list_remote

param_username = ''
if 'JUPYTERHUB_USER' in os.environ:
    param_username = os.environ['JUPYTERHUB_USER']
    
conf_remote_path_root = '/webdav/LAZ'
conf_remote_path_split = pathlib.Path(conf_remote_path_root + '/split_'+param_username)
conf_remote_path_retiled = pathlib.Path(conf_remote_path_root + '/retiled_'+param_username)
conf_remote_path_norm = pathlib.Path(conf_remote_path_root + '/norm_'+param_username)
conf_remote_path_targets = pathlib.Path(conf_remote_path_root + '/targets_'+param_username)
conf_remote_path_geotiffs = pathlib.Path(conf_remote_path_root + '/geotiffs_'+conf_username)
conf_local_tmp = pathlib.Path('/tmp')
conf_remote_path_ahn = conf_remote_path_root


param_hostname = 'https://lfw-ds001-i022.lifewatch.dev:32443/'
param_login = '20BNXDdL8mg24OaD'
param_password = 'zDoy0hNKkcnsdsQ@OYAVd'

conf_feature_name = 'perc_95_normalized_height'
conf_validate_precision = '0.001'
conf_tile_mesh_size = '10.'
conf_filter_type= 'select_equal'
conf_attribute = 'raw_classification'
conf_min_x = '-113107.81'
conf_max_x = '398892.19'
conf_min_y = '214783.87'
conf_max_y = '726783.87'
conf_n_tiles_side = '512'
conf_apply_filter_value = '1'
conf_laz_compression_factor = '7'
conf_max_filesize = '262144000'  # desired max file size (in bytes)

conf_wd_opts = { 'webdav_hostname': param_hostname, 'webdav_login': param_login, 'webdav_password': param_password}

## Fetching Laz Files from remote WebDAV

In [3]:
# Retrive Laz Files
conf_wd_opts = { 'webdav_hostname': param_hostname, 'webdav_login': param_login, 'webdav_password': param_password}
laz_files = [f for f in list_remote(get_wdclient(conf_wd_opts), pathlib.Path(conf_remote_path_ahn).as_posix())
             if f.lower().endswith('.laz')]

## Splitting big files into smaller files before retiling
This step can be added if the original files are too large for normal VMs to process

In [4]:
# Split big files
import numpy as np

def save_chunk_to_laz_file(in_filename, 
                           out_filename, 
                           offset, 
                           n_points):
    """Read points from a LAS/LAZ file and write them to a new file."""
    
    points = np.array([])
    
    with laspy.open(in_filename) as in_file:
        with laspy.open(out_filename, 
                        mode="w", 
                        header=in_file.header) as out_file:
            in_file.seek(offset)
            points = in_file.read_points(n_points)
            out_file.write_points(points)
    return len(points)

def split_strategy(filename, max_filesize):
    """Set up splitting strategy for a LAS/LAZ file."""
    with laspy.open(filename) as f:
        bytes_per_point = (
            f.header.point_format.num_standard_bytes +
            f.header.point_format.num_extra_bytes
        )
        n_points = f.header.point_count
    n_points_target = int(
        max_filesize * int(conf_laz_compression_factor) / bytes_per_point
    )
    stem, ext = os.path.splitext(filename)
    return [
        (filename, f"{stem}-{n}{ext}", offset, n_points_target)
        for n, offset in enumerate(range(0, n_points, n_points_target))
    ]

from webdav3.client import Client

client = Client(conf_wd_opts)
client.mkdir(conf_remote_path_split.as_posix())


remote_path_split = conf_remote_path_split

for file in laz_files:
    print('Splitting: '+file)
    client.download_sync(remote_path=os.path.join(conf_remote_path_ahn,file), local_path=file)
    inps = split_strategy(file, int(conf_max_filesize))
    for inp in inps:
        save_chunk_to_laz_file(*inp)
    client.upload_sync(remote_path=os.path.join(conf_remote_path_split,file), local_path=file)

    for f in os.listdir('.'):
        if not f.endswith('.LAZ'):
            continue
        os.remove(os.path.join('.', f))
        
remote_path_retiled = str(conf_remote_path_retiled)

Splitting: C_01GZ2.LAZ
Splitting: C_02CZ1.LAZ
Splitting: C_01GN1.LAZ
Splitting: C_01GN2.LAZ


In [5]:
# Read splitted Laz Files
remote_path_retiled

split_laz_files = [f for f in list_remote(get_wdclient(conf_wd_opts), pathlib.Path(conf_remote_path_ahn).as_posix())
             if f.lower().endswith('.laz')]

## Retiling of big files into smaller tiles

In [ ]:
# Retiling

grid_retile = {
    'min_x': float(conf_min_x),
    'max_x': float(conf_max_x),
    'min_y': float(conf_min_y),
    'max_y': float(conf_max_y),
    'n_tiles_side': int(conf_n_tiles_side)
}

retiling_input = {
    'setup_local_fs': {'tmp_folder': conf_local_tmp.as_posix()},
    'pullremote': conf_remote_path_split.as_posix(),
    'set_grid': grid_retile,
    'split_and_redistribute': {},
    'validate': {},
    'pushremote': conf_remote_path_retiled.as_posix(),
    'cleanlocalfs': {}
}

for file in split_laz_files:
    print('Retiling: '+file)
    retiler = Retiler(file.replace('"',''),label=file).config(retiling_input).setup_webdav_client(conf_wd_opts)
    retiler_output = retiler.run()
    
    
remote_path_retiled = conf_remote_path_retiled.as_posix()
print(type(remote_path_retiled))

2023-04-08 11:19:54,204 -           laserfarm.pipeline_remote_data -       INFO - Input dir set to /tmp/C_01GZ2.LAZ_input
2023-04-08 11:19:54,206 -           laserfarm.pipeline_remote_data -       INFO - Output dir set to /tmp/C_01GZ2.LAZ_output
2023-04-08 11:19:54,207 -           laserfarm.pipeline_remote_data -       INFO - Pulling from WebDAV /webdav/LAZ/split_spiros.koulouzis@lifewatch.eu/C_01GZ2.LAZ ...


Retiling: C_01GZ2.LAZ


2023-04-08 11:20:04,049 -           laserfarm.pipeline_remote_data -       INFO - ... pulling completed.
2023-04-08 11:20:04,051 -                        laserfarm.retiler -       INFO - Setting up the target grid
2023-04-08 11:20:04,053 -                        laserfarm.retiler -       INFO - Splitting file /tmp/C_01GZ2.LAZ_input/C_01GZ2.LAZ with PDAL ...
2023-04-08 11:20:33,565 -                        laserfarm.retiler -       INFO - ... splitting completed.
2023-04-08 11:20:33,568 -                        laserfarm.retiler -       INFO - Redistributing files to tiles ...
2023-04-08 11:20:33,573 -                        laserfarm.retiler -       INFO - ... file C_01GZ2_2.LAZ to tile_278_391
2023-04-08 11:20:33,574 -                        laserfarm.retiler -       INFO - ... file C_01GZ2_5.LAZ to tile_283_391
2023-04-08 11:20:33,576 -                        laserfarm.retiler -       INFO - ... file C_01GZ2_3.LAZ to tile_279_390
2023-04-08 11:20:33,577 -                        laser

Retiling: C_02CZ1.LAZ


2023-04-08 11:21:09,673 -           laserfarm.pipeline_remote_data -       INFO - ... pulling completed.
2023-04-08 11:21:09,678 -                        laserfarm.retiler -       INFO - Setting up the target grid
2023-04-08 11:21:09,681 -                        laserfarm.retiler -       INFO - Splitting file /tmp/C_02CZ1.LAZ_input/C_02CZ1.LAZ with PDAL ...
2023-04-08 11:21:45,434 -                        laserfarm.retiler -       INFO - ... splitting completed.
2023-04-08 11:21:45,438 -                        laserfarm.retiler -       INFO - Redistributing files to tiles ...
2023-04-08 11:21:45,443 -                        laserfarm.retiler -       INFO - ... file C_02CZ1_7.LAZ to tile_296_391
2023-04-08 11:21:45,445 -                        laserfarm.retiler -       INFO - ... file C_02CZ1_5.LAZ to tile_295_390
2023-04-08 11:21:45,447 -                        laserfarm.retiler -       INFO - ... file C_02CZ1_3.LAZ to tile_294_390
2023-04-08 11:21:45,448 -                        laser

In [ ]:
print(retiler_output)

In [ ]:
# Retrive retiled tiles
remote_path_retiled

tiles = [t.strip('/') for t in list_remote(get_wdclient(conf_wd_opts), conf_remote_path_retiled.as_posix())
         if fnmatch.fnmatch(t, 'tile_*_*/')]

In [ ]:
# Feature Extraction
    
for t in tiles:
    features = [conf_feature_name]

    tile_mesh_size = float(conf_tile_mesh_size)

    grid_feature = {
        'min_x': float(conf_min_x),
        'max_x': float(conf_max_x),
        'min_y': float(conf_min_y),
        'max_y': float(conf_max_y),
        'n_tiles_side': int(conf_n_tiles_side)
    }

    feature_extraction_input = {
        'setup_local_fs': {'tmp_folder': conf_local_tmp.as_posix()},
        'pullremote': conf_remote_path_retiled.as_posix(),
        'load': {'attributes': [conf_attribute]},
        'normalize': 1,
        'apply_filter': {
            'filter_type': conf_filter_type, 
            'attribute': conf_attribute,
            'value': [int(conf_apply_filter_value)]#ground surface (2), water (9), buildings (6), artificial objects (26), vegetation (?), and unclassified (1)
        },
        'generate_targets': {
            'tile_mesh_size' : tile_mesh_size,
            'validate' : True,
            'validate_precision': float(conf_validate_precision),
            **grid_feature
        },
        'extract_features': {
            'feature_names': features,
            'volume_type': 'cell',
            'volume_size': tile_mesh_size
        },
        'export_targets': {
            'attributes': features,
            'multi_band_files': False
        },
        'pushremote': conf_remote_path_targets.as_posix(),
    #     'cleanlocalfs': {}
    }
    idx = (t.split('_')[1:])

    processing = DataProcessing(t, tile_index=idx,label=t).config(feature_extraction_input).setup_webdav_client(conf_wd_opts)
    processing.run()
    
remote_path_targets = conf_remote_path_targets.as_posix()

print(type(remote_path_targets))

2023-04-08 11:25:28,748 -                laserfarm.data_processing -       INFO - ... normalization completed.
2023-04-08 11:25:28,751 -                laserfarm.data_processing -       INFO - Filtering point-cloud data
2023-04-08 11:25:28,953 -                laserfarm.data_processing -       INFO - Setting up the target grid
2023-04-08 11:25:28,955 -                laserfarm.data_processing -       INFO - Checking whether points belong to cell (278,391)
2023-04-08 11:25:28,968 -                laserfarm.data_processing -       INFO - Generating target point mesh with 10.0m spacing 
2023-04-08 11:25:28,971 -                laserfarm.data_processing -       INFO - Building volume of type cell
2023-04-08 11:25:28,972 -                laserfarm.data_processing -       INFO - Constructing neighborhoods
2023-04-08 11:25:28,973 -                laserfarm.data_processing -       INFO - Starting feature extraction ...
2023-04-08 11:25:28,974 -                                     root -       

In [ ]:
# GeoTIFF Export

remote_path_targets

#feature = features

# setup input dictionary to configure the GeoTIFF export pipeline
geotiff_export_input = {
    'setup_local_fs': {'tmp_folder': conf_local_tmp.as_posix()},
    'pullremote': conf_remote_path_targets.as_posix(),
    'parse_point_cloud': {},
    'data_split': {'xSub': 1, 'ySub': 1},
    'create_subregion_geotiffs': {'output_handle': 'geotiff'},
    'pushremote': conf_remote_path_geotiffs.as_posix(),
    'cleanlocalfs': {}   
}

writer = GeotiffWriter(input_dir=conf_feature_name, bands=conf_feature_name, label=conf_feature_name).config(geotiff_export_input).setup_webdav_client(conf_wd_opts)
writer.run()

In [3]:
import os

if 'JUPYTERHUB_USER' in os.environ:
    param_username = os.environ['JUPYTERHUB_USER']
param_username

'spiros.koulouzis@lifewatch.eu'